In [1]:
import pandas as pd, json, numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
%matplotlib inline

Load airports of each country

In [2]:
L=json.loads(file('../json/L.json','r').read())
M=json.loads(file('../json/M.json','r').read())
N=json.loads(file('../json/N.json','r').read())

In [3]:
import requests

In [4]:
AP={}
for c in M:
    if c not in AP:AP[c]={}
    for i in range(len(L[c])):
        AP[c][N[c][i]]=L[c][i]

record schedules for 2 weeks, then augment count with weekly flight numbers.
seasonal and seasonal charter will count as once per week for 3 months, so 12/52 per week. TGM separate, since its history is in the past.

parse Departures

In [6]:
baseurl='https://www.airportia.com/'
import requests, urllib2

In [7]:
def urlgetter(url):
    s = requests.Session()
    cookiesopen = s.get(url)
    cookies=str(s.cookies)
    fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
    #push token
    opener = urllib2.build_opener()
    for k in fcookies:
        opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
    #read html
    return s.get(url).content

good dates

In [9]:
SC={}

In [10]:
for h in range(len(AP.keys())):
    c=AP.keys()[h]
    #country not parsed yet
    if c not in SC:
        SC[c]=[]
        print h,c
        airportialinks=AP[c]
        #all airports of country
        for i in airportialinks:
            print i,
            url=baseurl+airportialinks[i]
            m=urlgetter(url)
            soup = BeautifulSoup(m, "lxml")
            #if there are flights at all
            if len(soup.findAll('table'))>0:
                good=False
                w=pd.read_html(m)[0]
                #if there are daily flights
                if 'Date' not in w.columns:good=True
                #if there are flights at least in the last 2w
                elif 5>int(w.loc[0]['Date'][-1])>3:good=True 
                if good:
                    SC[c].append(i)
                else: print '--2W,',
            else: print '--NO,', 
    print

0 Canada
YUY YUX XBW --NO, ZQS --NO, XBR --NO, YUL XBE YUB YUD XBB --NO, YRT ZTS --NO, XPK --NO, YUT HZP --2W, XPP --NO, LAK ZFN YBX YBY --2W, YBQ --NO, YBR YBS --2W, YBT CJH --NO, YBV YBW --NO, YBH --NO, YBI --2W, YBJ --NO, YBK YBL YBM --NO, YBN --NO, YBO --NO, YBA --NO, YBB YBC YBD --NO, YLL YBF YBG IUM --NO, ZGR ZGS --NO, XKO --NO, XKS YYN --NO, ZGI AKV XFZ --NO, YCE --NO, SYF --NO, YIG --NO, YIF YIB --NO, YIO YIK YIV XTL ZMT YFE --NO, YFG --NO, YFA YFB YFC YFL --NO, YFO YFH YFJ YFR --NO, YFS YFX --2W, YPR YPS --NO, YPP --NO, YPQ ZRR --NO, YPT --NO, YPZ --NO, YPX YPY YPB YPC YPA YPF --NO, YPG --NO, YPD --NO, YPE --2W, YPJ YPH YPI --NO, YPN YPO YPL YPM CFQ --NO, ZNA ZNG --NO, ZJN --NO, ZNU --NO, XQU YHB --NO, YMQ --NO, YMP --NO, YMR --NO, YMT YMW --NO, YMX YMA --NO, YMC --NO, QBC YME --NO, YMD --NO, YMG --NO, YMF --NO, YMI --NO, YMH --2W, YMJ --NO, YMM YML --NO, YMO YMN ZAC SUR YSK YSJ YSI --NO, YSH --NO, YSO YSN --NO, YSM YSL --NO, YSC --NO, XLF --NO, YSA --NO, YSG XLB YSE --NO, YSD

Save

In [11]:
import json
file('../json/SC.json','w').write(json.dumps(SC))

Reparse

In [12]:
for h in range(len(AP.keys())):
    c=AP.keys()[h]
    #country not parsed yet
    print h,c
    airportialinks=AP[c]
    #all airports of country
    for i in airportialinks:
        if i not in SC[c]:
            print i,
            url=baseurl+airportialinks[i]
            m=urlgetter(url)
            soup = BeautifulSoup(m, "lxml")
            #if there are flights at all
            if len(soup.findAll('table'))>0:
                good=False
                w=pd.read_html(m)[0]
                #if there are daily flights
                if 'Date' not in w.columns:good=True
                #if there are flights at least in the last 2w
                elif 5>int(w.loc[0]['Date'][-1])>3:good=True 
                if good:
                    SC[c].append(i)
                else: print '--2W,',
            else: print '--NO,', 
    print

0 Canada
XBW --NO, ZQS --NO, XBR --NO, XBB --NO, ZTS --NO, XPK --NO, HZP --2W, XPP --NO, YBY --2W, YBQ --NO, YBS --2W, CJH --NO, YBW --NO, YBH --NO, YBI --2W, YBJ --NO, YBM --NO, YBN --NO, YBO --NO, YBA --NO, YBD --NO, IUM --NO, ZGS --NO, XKO --NO, YYN --NO, XFZ --NO, YCE --NO, SYF --NO, YIG --NO, YIB --NO, YFE --NO, YFG --NO, YFL --NO, YFR --NO, YFX --2W, YPS --NO, YPP --NO, ZRR --NO, YPT --NO, YPZ --NO, YPF --NO, YPG --NO, YPD --NO, YPE --2W, YPI --NO, CFQ --NO, ZNG --NO, ZJN --NO, ZNU --NO, YHB --NO, YMQ --NO, YMP --NO, YMR --NO, YMW --NO, YMA --NO, YMC --NO, YME --NO, YMD --NO, YMG --NO, YMF --NO, YMI --NO, YMH --2W, YMJ --NO, YML --NO, YSI --NO, YSH --NO, YSN --NO, YSL --NO, YSC --NO, XLF --NO, YSA --NO, YSE --NO, YSD --NO, YSZ --NO, YSX --NO, YSS --NO, YSR --NO, YSQ --NO, YSP --NO, YSV --NO, YZU --2W, YZW --NO, YZX --NO, YZA --NO, YZC --NO, YZE --NO, YZH --NO, YZM --NO, YOD --NO, YTX --NO, YTT --NO, YTU --NO, YTP --NO, YTN --NO, YTO --NO, YTM --2W, YTJ --NO, YTK --NO, YTI --NO, Y

In [13]:
file('../json/SC2.json','w').write(json.dumps(SC))